# Installation H2 

Le système utilisé pendant les TME est H2.

In [1]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Télécharger le pilote de H2

In [2]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-05-01 14:53:40--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1,0M) [application/octet-stream]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    in 0,01s   

2024-05-01 14:53:40 (104 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]

--2024-05-01 14:53:41--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2543011 (2,4M) [application/octet-stream]
Saving to: ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  --.-KB/s    in 0,02s   

2024-05-01 14:53:4

**Relancez le kernel**: Kernel -> Restart kernel ...

https://www.psycopg.org/docs/
http://localhost:8082

In [3]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['jo5613.mv.db']

## Démarrer le serveur SQL H2

In [4]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = '06'
#BB=numÉro binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = '13'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5613


In [5]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

# Fonctions utiles

## connect_H2

In [6]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [7]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount:
            names = [desc[0] for desc in cursor.description]
    
            lengths={}
            for attr in names:
                lengths[attr]=len(attr)
            for ligne in cursor:
                i=0
                for attr in ligne:
                    lengths[names[i]]=max(lengths[names[i]],len(str(attr)))
                    i=i+1
            print('|',end='')
            for attr in names:
                print(str(attr).ljust(lengths[attr]),end='|')
            print()
            print('|',end='')
            for attr in names:
                print(''.ljust(lengths[attr]+1,'-'),end='')
            print()
            cursor.execute(query)
            for ligne in cursor:
                i=0
                print('|',end='')
                for attr in ligne:
                    print(str(attr)[:lengths[names[i]]].ljust(lengths[names[i]]),end='|')
                    i=i+1
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

**TD/TME 7 : REQUÊTES D'AGRÉGATION ET DIVISION**

Ce TD/TME utilise les données contenues dans le fichier **Base Astronomie_H2.sql**

**Attention**: vous pouvez cliquer sur les 3 lignes dans la fenêtre de gauche pour d'afficher les différentes sections du notebook 

# TD7:  REQUÊTES D'AGRÉGATION ET DIVISION

On veut réaliser une application pour des données astronomiques. Les observations, les dates et les
valeurs observées sont inventées pour les besoins de l’exercice. Le rayon des astres est en km. On
suppose que les attributs ne peuvent pas être null. On considère le schéma suivant :


**Categorie**(<u>idC</u>,nom)<br/>
**Astre**(<u>idA</u>,nom,rayon,idC*) avec idC référence Categorie(idC)<br/>
**TourneAutour**(<u>idA1*</u>,idA2*,position) avec idA1 et idA2 référence Astre(idA)<br/>
**Observation**(<u>idO</u>,idA*,dateObs,valObs) avec idA référence Astre(idA)<br/>

Exemples de données :
 - La Terre est la troisième planète qui tourne autour du Soleil.<br/>
 - Le 10 mai 2010, on a réalisé deux observations du Soleil avec les valeurs 12001 et 12003.


## Connexion à H2

In [8]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection to H2 DB successful


## Créer les tables et charger les données Base Astronomie_H2.sql

In [9]:
schemafile=open("Base Astronomie_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


<u>**B - Requêtes: Fonctions d’agrégation « COUNT, SUM, AVG, MIN, MAX »**</u>

## **Q5.**. 
Le nombre de catégories. Résultat : 3

In [10]:
query="""
select count(*) as nbCategorie
from categorie

"""
execute(connection,query,show=True)

1 rows
|nbcategorie|
|------------
|3          |


<cursor object at 0x7f198427c220; closed: -1>

 ## **Q6.** 
Le nombre de catégories pour lesquelles on connaît au moins un astre. Résultat : 3

In [11]:
query="""
select count(*) as nbCategorie
from categorie c
where exists (select * from astre a where a.idc = c.idc)
"""
execute(connection,query,show=True)

1 rows
|nbcategorie|
|------------
|3          |


<cursor object at 0x7f198427cd60; closed: -1>

##  **Q7.**) 
Le rayon minimal et le rayon maximal de tous les astres. Résultat : (1737, 696342)

In [12]:
query="""
select min(a.rayon) as min, max(a.rayon) as max
from astre a

"""
execute(connection,query,show=True)

1 rows
|min |max   |
|------------
|1737|696342|


<cursor object at 0x7f198427cb80; closed: -1>

## **Q8.** 
Le rayon moyen en milliers de km (et non pas en km), arrondi à 2 chiffres après la virgule.
Résultat : ‘176,96 milliers km’. 

Aide : utilisez :

a) round(valeur,nb) pour garder seulement nb décimales à valeur

b) l’opérateur de concaténation || pour concaténer des chaînes, attributs… :
select ‘abc’ || attribut || ‘def’ from UneTable ;

In [13]:
query="""
select avg(a.rayon)/1000 as moyen
from astre a

"""
execute(connection,query,show=True)

1 rows
|moyen |
|-------
|176.96|


<cursor object at 0x7f198427c9a0; closed: -1>

## **Q9.a)** 
Le rayon de l'astre de rayon maximal. Résultat : 696342

In [14]:
query="""
select max(a.rayon) as max
from astre a
"""
execute(connection,query,show=True)

1 rows
|max   |
|-------
|696342|


<cursor object at 0x7f198427c130; closed: -1>

##  **Q9.b)** 
Le nom et le rayon de l'astre de rayon maximal. Résultat : (Soleil, 696342)

In [15]:
query="""
select a.nom, a.rayon
from astre a
where a.rayon = (select max(a.rayon) as max from astre a)
"""
execute(connection,query,show=True)

1 rows
|nom   |rayon |
|--------------
|Soleil|696342|


<cursor object at 0x7f198427d030; closed: -1>

<u>**C - Requêtes: Partitionnement « group by »**</u>

##  **Q10.** 
Pour chaque catégorie, l'identifiant de la catégorie et le nombre d'astres de cette catégorie
(une seule table suffit), ordonner par nombre d’astres décroissant.
Résultats (3 lignes) : (11, 2) ; (10,1) ; (12,1)

In [16]:
query="""
select c.nom, count(*) as nbAstre
from categorie c, astre a
where a.idc = c.idc
group by c.nom

"""
execute(connection,query,show=True)

3 rows
|nom      |nbastre|
|------------------
|planète  |2      |
|satellite|1      |
|étoile   |1      |


<cursor object at 0x7f198427d210; closed: -1>

##  **Q11.a)** 
Pour chaque astre, l’idA de l’astre et le nombre d'astres qui lui tournent autour. Pour
simplifier, prendre que les astres qui ont au moins un autre astre qui leur tourne autour.
Résultats (2 lignes) : (100, 2) ; (101,1).

In [17]:
query="""
select a.ida, count(*)
from astre a, tourneAutour tA
where tA.ida2 = a.ida
group by a.ida

"""
execute(connection,query,show=True)

2 rows
|ida|?column?|
|-------------
|100|2       |
|101|1       |


<cursor object at 0x7f198427ce50; closed: -1>

##  **Q11. b)** 
Même question, mais on veut le nom de l’astre au lieu de l’idA.
Résultats (2 lignes) : (Soleil, 2) ; (Terre, 1) 

In [18]:
query="""
select a.nom, count(*)
from astre a, tourneAutour tA
where tA.ida2 = a.ida
group by a.ida

"""
execute(connection,query,show=True)


2 rows
|nom   |?column?|
|----------------
|Soleil|2       |
|Terre |1       |


<cursor object at 0x7f198427d300; closed: -1>

##  **Q12.** 
Pour chaque couple (astre,date), le nom de l'astre, la date de l'observation et la moyenne des
valeurs des observations réalisées.
Résultats : (Soleil,2010-05-10,12002) ; (Terre,2013-12-18,8005) ; (Lune,2014-08-27)

In [19]:
query="""
select a.nom, o.dateobs, avg(o.valobs)
from astre a, observation o
where o.ida = a.ida
group by a.nom, o.dateobs
"""
execute(connection,query,show=True)

3 rows
|nom   |dateobs   |?column?|
|---------------------------
|Lune  |2014-08-27|3007.0  |
|Soleil|2010-05-10|12002.0 |
|Terre |2013-12-18|8005.0  |


<cursor object at 0x7f198427d3f0; closed: -1>

##  **Q13.a)**  
Pour chaque catégorie, l'idC de la catégorie et le rayon maximal des astres de cette
catégorie, ordonner par rayon croissant. Aide : une seule table suffit.
Résultats (3 lignes) : (12,1737) ; (11,6371) ; (10,696342)

In [20]:
query="""
select a.idc, max(rayon) as max
from astre a
group by a.idc
order by max
"""
execute(connection,query,show=True)

3 rows
|idc|max   |
|-----------
|12 |1737  |
|11 |6371  |
|10 |696342|


<cursor object at 0x7f198427d4e0; closed: -1>

##  **Q13.b)**  
On veut maintenant afficher non pas l’idC, mais le nom de la catégorie.
Pour chaque catégorie, le nom de la catégorie et le rayon maximal des astres de cette
catégorie, ordonner par nombre d’astres dans chaque catégorie décroissant.
Résultats (3 lignes) : (planète, 6371) ; (étoile,696342) ; (satellite, 1737)

In [21]:
query="""
select c.nom, max(rayon) as max
from astre a, categorie c
where a.idc = c.idc 
group by c.idc
order by count(*) desc

"""
execute(connection,query,show=True)

3 rows
|nom      |max   |
|-----------------
|planète  |6371  |
|étoile   |696342|
|satellite|1737  |


<cursor object at 0x7f198427cf40; closed: -1>

##  **Q13.c)**  
On veut maintenant afficher non pas le nom de la catégorie, mais le nom de l’astre.
Pour chaque catégorie, l'idC de la catégorie, le(s) nom(s) de(s) astre(s) de rayon
maximal et le rayon maximal des astres de cette catégorie. Aide : deux tables suffisent.
Résultats (3 lignes) : (10, Soleil,696342) ; (11,Terre,6371) ; (12, Lune, 1737)

In [22]:
query="""
select a.idc, a.nom, a.rayon
from astre a
where not exists (select * from astre a2 where a2.idc = a.idc and a2.rayon>a.rayon)
"""
execute(connection,query,show=True)

3 rows
|idc|nom   |rayon |
|------------------
|10 |Soleil|696342|
|11 |Terre |6371  |
|12 |Lune  |1737  |


<cursor object at 0x7f198427d5d0; closed: -1>

<u>**D - Requêtes: Partitionnement avec sélection de partition « group by / having »**</u>

## **Q14.** 
L'idC des catégories pour lesquelles il existe au moins deux astres dans cette catégorie.
Résultat : 11

In [23]:
query="""
select a.idc
from astre a
group by a.idc
having count(*)>1
"""
execute(connection,query,show=True)

1 rows
|idc|
|----
|11 |


<cursor object at 0x7f198427d7b0; closed: -1>

## **Q15.** 
Pour chaque date (où il y a eu au moins une observation) et où il y a eu une valeur maximale
observée supérieure à 8000, la date, le nombre d'observations effectuées ce jour là et la
valeur maximale observée. Résultats(2 lignes):(2010-05-10,2, 12003);(2013-12-18, 1, 8005)

In [24]:
query="""
select o.dateobs, count(*) as nb_obs, max(o.valobs) as max
from observation o
group by o.dateobs
having max(o.valobs)>=8000
"""
execute(connection,query,show=True)

2 rows
|dateobs   |nb_obs|max  |
|------------------------
|2010-05-10|2     |12003|
|2013-12-18|1     |8005 |


<cursor object at 0x7f198427c310; closed: -1>

##  **Q16.** 
 Le nom des astres pour lesquels on a fait exactement deux observations. Résultat : Soleil

In [25]:
query="""
select a.nom
from astre a, observation o
where o.ida =a.ida
group by a.ida
having count(*)=2
"""
execute(connection,query,show=True)

1 rows
|nom   |
|-------
|Soleil|


<cursor object at 0x7f198427d8a0; closed: -1>

##  **Q17.** 
Le nom des astres qui ont au moins deux astres qui leur tournent autour. Résultat : Soleil

In [26]:
query="""
select a.nom
from astre a, tourneAutour tA
where tA.ida2 = a.ida
group by a.ida
having count(*)>1
"""
execute(connection,query,show=True)

1 rows
|nom   |
|-------
|Soleil|


<cursor object at 0x7f198427db70; closed: -1>

<u>**E - Requêtes: Réécriture double négation avec Group By et Having**</u>

##  **Q18.a)** 
 Les dates d’observations pour lesquelles il y a au moins une observation pour chaque astre existant dans la bases de données.
Résultat : aucun sur cet exemple

In [27]:
query="""
select o.dateobs
from observation o
group by o.dateobs, o.ida
having count(*) = (select count(*) from astre)

"""
execute(connection,query,show=True)

0 rows


<cursor object at 0x7f198427dc60; closed: -1>

##  **Q18.b)** 
Les dates d’observations pour lesquelles il y a au moins une observation pour chaque astre observés au moins une fois.
Résultat : aucun sur cet exemple

In [28]:
query="""
select o.dateobs
from observation o
group by o.dateobs, o.ida
having count(*) = (select count(*) from astre a where exists (select * from observation o2 where o2.ida = a.ida))
"""
execute(connection,query,show=True)


0 rows


<cursor object at 0x7f198427dd50; closed: -1>

##  **Q19.a)** 
L'idC des catégories pour lesquelles il existe au moins une observation pour chaque astre
de cette catégorie. Résultats (2 lignes) : 10 ; 12

In [29]:
query="""
select a.idc
from astre a
group by a.idc
having count(a.ida)=(select count(distinct o.ida) from astre a2, observation o where a2.ida = o.ida and a2.idc = a.idc)
"""
execute(connection,query,show=True)

2 rows
|idc|
|----
|10 |
|12 |


<cursor object at 0x7f198427de40; closed: -1>

##  **Q19.b)**  
Même question mais on veut afficher non pas l’idC, mais le nom de la catégorie.
Résultats (2 lignes) : étoile ; satellite

In [30]:
query="""
select c.nom
from astre a, categorie c
where c.idc = a.idc
group by a.idc
having count(a.ida)=(select count(distinct o.ida) from astre a2, observation o where a2.ida = o.ida and a2.idc = a.idc)

"""
execute(connection,query,show=True)

2 rows
|nom      |
|----------
|étoile   |
|satellite|


<cursor object at 0x7f198427df30; closed: -1>

# TME7: REQUÊTES D'AGRÉGATION ET DIVISION

On reprend le schéma schéma « Jeux Olympiques d'hiver 2014 ».

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>

## Créer les tables et charger les données bd-jo-v2_H2.sql

In [31]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


<u>**Requêtes: Fonctions d’agrégation « COUNT, SUM, AVG, MIN, MAX »**</u>

##  **Q1.**  
Le nombre d’athlètes.
Résultat (1 ligne) : 2431

In [32]:
query="""
select count(*)
from athlete

"""
execute(connection,query,show=True)

1 rows
|?column?|
|---------
|2431    |


<cursor object at 0x7f198427e020; closed: -1>

##  **Q2.**  
Le nombre d’athlètes ayant participé à au moins une épreuve en individuel.
Résultat (1 ligne) : 1558

In [33]:
query="""
select count(*)
from athlete a
where a.aid = any (select ri.aid from rangIndividuel ri)

"""
execute(connection,query,show=True)

1 rows
|?column?|
|---------
|1558    |


<cursor object at 0x7f198427d6c0; closed: -1>

##  **Q3.**  
L'âge moyen des sportifs dont le code pays est 'FRA' (France) au 06/02/2014.
Résultat (1 ligne) : 26,8

Aide :

– utilisez round(valeur,nb) pour garder seulement nb décimales à valeur

In [34]:
query="""
select round(avg(datediff(day,a.datenaissance, '2014-02-06'))/365,1) as age_moyen
from athlete a
where a.datenaissance is not null and a.codepays= 'FRA'
"""
execute(connection,query,show=True)

1 rows
|age_moyen|
|----------
|26.8     |


<cursor object at 0x7f198427e200; closed: -1>

##  **Q4.**  
La durée moyenne, minimale et maximale des épreuves.
Résultat (1 ligne) : « Durée moyenne = 1,98 min = 1 max = 16 »

Aide : utilisez l’opérateur de concaténation ||

Attention : entre le 10/01/2014 et le 13/01/2014, il y a une durée de 4 jours (et non pas 3).

In [35]:
query="""
select  
'Durée moyenne = ' || round ( avg(datediff(day, ep.datedebut, ep.datefin)+1), 2), 
'min = ' || min(datediff(day, ep.datedebut, ep.datefin)+1), 
'max = ' ||max(datediff(day, ep.datedebut, ep.datefin)+1)
from epreuve ep

"""
execute(connection,query,show=True)

1 rows
|?column?            |?column?            |?column?            |
|---------------------------------------------------------------
|Durée moyenne = 1.98|min = 1             |max = 16            |


<cursor object at 0x7f198427e110; closed: -1>

## **Q5.** 
Le nombre moyen d'athlètes par pays, c'est-à-dire le nombre d'athlètes divisé par le nombre
de pays (ayant au moins un athlète). Résultat (1 ligne) : 27,625

In [36]:
query="""
select round (count(*)*1.0/(select count(*) from pays where codepays = any (select codepays from athlete)), 3)
from athlete a
"""
execute(connection,query,show=True)

1 rows
|round |
|-------
|27.625|


<cursor object at 0x7f198427e4d0; closed: -1>

<u>**Partitionnement « group by »**</u>

## **Q6.** 
Pour chaque pays, le nom du pays et le nombre d’athlètes, ordonner par nombre d’athlètes
croissant.
Résultats (88 lignes) : (PAK,1) ; (HKG, 1) ; ... ; (USA, 196) ; (CAN,221)

In [38]:
query="""
select a.codepays, count(*)
from athlete a
group by a.codepays
order by 2
"""
execute(connection,query,show=False)

88 rows


<cursor object at 0x7f198427e7a0; closed: -1>

## **Q7.** 
Le nombre moyen d'athlètes par pays (avec group by). Aide : compter le nombre
d’athlètes dans chaque pays (ayant au moins un athlète) avec une sous-requêtes dans la clause FROM, puis faire la moyenne.
Résultat (1 ligne) : 27,625

In [46]:
query="""
select round(avg(nb), 3)
from (select count(*) as nb from Athlete group by codePays)
"""
execute(connection,query,show=True)

1 rows
|round |
|-------
|27.625|


<cursor object at 0x7f198427d120; closed: -1>

## **Q8.** 
Pour chaque équipe, l’eqid de l'équipe et le nombre d'athlètes, ordonner par nombre
d’athlètes décroissant.
Résultats (296 lignes) : (164,25) ; (165,25) ; (166,25) ; ... ; (180,2) ; (181, 2) ; (182, 2)

In [50]:
query="""
select eqid, count(*)
from athletesEquipe
group by eqid
order by 2 desc
"""
execute(connection,query,show=False)

296 rows


<cursor object at 0x7f198427ec50; closed: -1>

## **Q9.** 
Pour chaque catégorie, la catégorie et le nombre d'épreuves.
Résultats (3 lignes) : (Femmes,43) ; (Mixte,6) ; (Hommes,49)

In [51]:
query="""
select categorie, count(*)
from epreuve
group by categorie
"""
execute(connection,query,show=True)

3 rows
|categorie|?column?|
|-------------------
|Femmes   |43      |
|Hommes   |49      |
|Mixte    |6       |


<cursor object at 0x7f198427eb60; closed: -1>

## **Q10.** 
Pour chaque sport, le nom du sport et le nombre d'épreuves, ordonner par nombre d'épreuves
décroissant.

Résultats (15 lignes) : (Patinage de vitesse,12) ; (Ski de fond,12) ; ... ;(Hockey sur glace,2)

In [52]:
query="""
select s.nomsp, count(*)
from sport s, epreuve ep
where s.sid = ep.sid
group by ep.sid
order by 2 desc
"""
execute(connection,query,show=True)

15 rows
|nomsp                               |?column?|
|----------------------------------------------
|Patinage de vitesse                 |12      |
|Ski de fond                         |12      |
|Biathlon                            |11      |
|Ski acrobatique                     |10      |
|Ski alpin                           |10      |
|Surf des neiges                     |10      |
|Patinage de vitesse sur piste courte|8       |
|Patinage artistique                 |5       |
|Luge                                |4       |
|Saut à ski                          |4       |
|Bobsleigh                           |3       |
|Combiné nordique                    |3       |
|Curling                             |2       |
|Hockey sur glace                    |2       |
|Skeleton                            |2       |


<cursor object at 0x7f198427f100; closed: -1>

## **Q11.** 
Pour chaque pays, le code du pays, le nombre de médailles en épreuve individuelle gagnées
et le nombre d'athlètes ayant gagnés au moins une médaille. Ordonner par nombre de
médailles décroissant. Aide : 2 tables seulement sont nécessaires.
Résultats (24 lignes) : (NOR, 24,19) ; (NED,22,15) ; ...

In [69]:
query="""
select a.codepays, count(*) as nb_medailles, count(distinct ri.aid) as nb_athletes
from athlete a, rangIndividuel ri
where ri.aid = a.aid and ri.rang < 4
group by a.codepays
order by 2 desc
"""
execute(connection,query,show=True)

24 rows
|codepays|nb_medailles|nb_athletes|
|----------------------------------
|NOR     |24          |19         |
|NED     |22          |15         |
|USA     |20          |20         |
|RUS     |19          |16         |
|CAN     |17          |16         |
|FRA     |14          |12         |
|AUT     |13          |11         |
|GER     |12          |11         |
|SLO     |10          |5          |
|CHN     |8           |8          |
|SUI     |8           |7          |
|SWE     |8           |7          |
|CZE     |7           |5          |
|JPN     |7           |7          |
|BLR     |6           |3          |
|ITA     |6           |4          |
|KOR     |6           |4          |
|POL     |4           |3          |
|AUS     |3           |3          |
|GBR     |2           |2          |
|CRO     |1           |1          |
|FIN     |1           |1          |
|KAZ     |1           |1          |
|LAT     |1           |1          |


<cursor object at 0x7f19842cc130; closed: -1>

## **Q12.** 
Pour chaque pays et sport, le code du pays, le sid du sport, le nombre de médailles en
épreuve individuelle gagnées, le nombre d'athlètes ayant gagnés au moins une médaille,
ordonner d'abord par code pays, puis par nombre de médailles décroissant.
Résultats (84 lignes) : (AUS,12,2,2); (AUS,15,1,1);(AUT,13,9,7);(AUT,15,2,2);...

In [72]:
query="""
select a.codepays, ep.sid, count(*) as nb_medailles, count(distinct ri.aid) as nb_athletes
from athlete a, rangIndividuel ri, epreuve ep
where ri.aid = a.aid and ri.rang < 4 and ep.epid = ri.epid
group by a.codepays, ep.sid
order by 1 , 3 desc
"""
execute(connection,query,show=False)

84 rows


<cursor object at 0x7f19842cc220; closed: -1>

<u>**Partitionnement avec « group by / having »**</u>

## **Q13.a)** 
L’eqid de la ou des équipes qui sont composées d'exactement 10 athlètes. Résultat (1 ligne) : 226

In [78]:
query="""
select eqid
from athletesEquipe
group by eqid
having count(*) = 10
"""
execute(connection,query,show=True)

1 rows
|eqid|
|-----
|226 |


<cursor object at 0x7f198427fc40; closed: -1>

## **Q13.b)** 
L’eqid de la ou des équipes qui sont composées du plus d’athlètes pour ces JO. Résultats (3 lignes) : 164 ; 165 ; 166

In [79]:
query="""
select eqid
from athletesEquipe
group by eqid
having count(*) >= all (select count(*) from athletesEquipe group by eqid)
"""
execute(connection,query,show=True)

3 rows
|eqid|
|-----
|164 |
|165 |
|166 |


<cursor object at 0x7f19842cc5e0; closed: -1>

## **Q14.** 
 Le nombre d'épreuves en individuel où il y a eu au moins 100 participants.
 Aide: calculer les epreuves dans une sous-requête dans la clause FROM 
Résultat (1 ligne ) : 2

In [81]:
query="""
select count(*)
from (select epid from rangIndividuel group by epid having count(*) > 99)
"""
execute(connection,query,show=True)

1 rows
|?column?|
|---------
|2       |


<cursor object at 0x7f19842cc7c0; closed: -1>

## **Q15.** 
Le nom des pays qui ont gagné au moins 20 médailles aux épreuves individuelles.
Résultats (3 lignes) : Pays-Bas ; États-Unis ; Norvège

In [82]:
query="""
select p.nomp
from pays p, rangIndividuel ri, athlete a
where ri.aid = a.aid and a.codepays = p.codepays and ri.rang < 4
group by a.codepays
having count(*) > 19
"""
execute(connection,query,show=True)

3 rows
|nomp      |
|-----------
|Pays-Bas  |
|Norvège   |
|États-Unis|


<cursor object at 0x7f19842cc8b0; closed: -1>

<u>**Division en SQL**</u>

## **Q16.** 
Le sid des sports qui ont des épreuves dans toutes les catégories existantes.
Résultats (3 lignes) : 1 ; 6 ; 7

Principe : pour chaque épreuve, on compte le nombre de catégories, puis on regarde si il est
égal au nombre total de catégories d’épreuves.

In [83]:
query="""
select ep.sid
from epreuve ep
group by ep.sid
having count(distinct categorie) = (select count(distinct categorie) from epreuve)

"""
execute(connection,query,show=True)

3 rows
|sid|
|----
|1  |
|6  |
|7  |


<cursor object at 0x7f19842cc9a0; closed: -1>

## **Q17.** 
Le nom des pays qui ont participé aux épreuves en individuel de tous les sports en individuel. Résultats (3 lignes) : (Russie,12) ; (États-Unis,12) ; (Italie,12)

In [84]:
query="""
select p.nomp
from pays p, athlete a, rangIndividuel ri, epreuve ep
where ri.epid = ep.epid and ri.aid = a.aid and a.codepays = p.codepays
group by a.codepays
having count(distinct ep.sid) = (select count(distinct sid) from epreuve ep, rangIndividuel ri where ri.epid = ep.epid)
"""
execute(connection,query,show=True)

3 rows
|nomp      |
|-----------
|Italie    |
|Russie    |
|États-Unis|


<cursor object at 0x7f19842cca90; closed: -1>

**<u>BASE FOOFLE</u>**

## Créer les tables et charger les données base_foofle_H2.sql

In [86]:
schemafile=open("base_foofle_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


## **Q1.** 
Pour chaque sponsor, le nom du sponsor, le nombre de joueurs sponsorisés, le montant total
des sommes versées, ordonner par sommes versées décroissantes.
Résultats (7 lignes) : (Adadis,6,2340) ; (Robek,5,1426) ;...

In [89]:
query="""
select nsp, count(njo), sum(somme)
from sponsorise
group by nsp
"""
execute(connection,query,show=True)

7 rows
|nsp       |?column?|?column?|
|-----------------------------
|Adadis    |6       |2340    |
|Air Monaco|2       |1285    |
|Alcampo   |4       |1252    |
|Carouf    |3       |853     |
|GPS       |2       |478     |
|Palasse   |2       |510     |
|Robek     |5       |1426    |


<cursor object at 0x7f1966345a80; closed: -1>

## **Q2.** 
Quelles équipes ont joué au moins dans 3 stades différents ?
Résultats (2 lignes) : Fortiches ; Direkt

In [92]:
query="""
select eq1
from match
group by eq1
having count(distinct st) = 3
"""
execute(connection,query,show=True)

2 rows
|eq1      |
|----------
|Direkt   |
|Fortiches|


<cursor object at 0x7f19663463e0; closed: -1>

## **Q3.** 
Quels sponsors sponsorisent exactement un joueur pour chaque équipe qu'il sponsorise ?
Résultats (2 lignes) : Air Monaco ; Palasse

In [95]:
query="""
select s.nsp
from sponsorise s, joueur j
where s.njo = j.njo
group by nsp
having count(distinct s.njo) = count(distinct j.eq)
"""
execute(connection,query,show=True)

2 rows
|nsp       |
|-----------
|Air Monaco|
|Palasse   |


<cursor object at 0x7f19663464d0; closed: -1>

## **Q4.** 
Quel est le nombre total de kilomètres parcourus par chaque équipe. On suppose qu’après
chaque match, chaque équipe se rend directement au stade où aura lieu son prochain match (d’après la date du match). Aide : il existe 2 matchs ordonnés par leur date pour la même équipe, mais il n’existe pas un 3ième match entre les dates des 2 matchs pour cette équipe. Résultats (4 lignes) : (Fortiches,516) ; (Direkt,671) ; (Piepla,124) ; (Sabar,360)

In [110]:
query="""
select m.eq1, sum(nbkm)
from match m, match m2, distance d        
where m.eq1 = m2.eq1
    and m.eq2 <> m2.eq2 
    and m2.datem < m.datem 
    and d.st1 = m.st
    and d.st2 = m2.st
    and not exists (
        select * 
        from match m3 
        where m.eq1 = m3.eq1 
            and m3.datem < m2.datem 
            and m3.datem > m.datem
    )

group by m.eq1
"""
execute(connection,query,show=True)

4 rows
|eq1      |?column?|
|-------------------
|Direkt   |701     |
|Fortiches|696     |
|Piepla   |248     |
|Sabar    |360     |


<cursor object at 0x7f1966347880; closed: -1>

# Fermer la connexion

In [ ]:
connection.commit() # implicit avec close
connection.close()

In [ ]:
connection